<a href="https://colab.research.google.com/github/IndiraGundavarapu01/Large-Language-Model-utilizing-GPT-Architecture-for-similar-Text-Generation-/blob/main/GPTLanguageModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
'''!pip install Functional
from torch.nn import functional as F'''

'!pip install Functional\nfrom torch.nn import functional as F'

In [9]:
'''import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'''




"import os\nos.environ['CUDA_LAUNCH_BLOCKING'] = '1"

In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

blockSize = 64
batchSize = 128
maxIters = 1000
learningRate = 3e-4
evalIters = 100 #reports the loss
nEmbd = 384 #no of total dimensions we want to capture from concatenating all the heads
n_head = 6 #heads in parallel
nLayer = 6 #decoder blocks
dropout = 0.2 #20% of neurons are dropped out

cuda


In [11]:
#dataloader
chs = ""
with open('/content/Picture of dorian gray.txt', 'r', encoding = 'utf-8') as f:
          text = f.read()
          chs = sorted(list(set(text))) #reading the file as text

print(len(text)) #length of file


429014


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
print(text[:300]) #first 300 characters


The Picture of Dorian Gray

by Oscar Wilde


THE PREFACE


The artist is the creator of beautiful things. To reveal art and
conceal the artist is art’s aim. The critic is he who can translate
into another manner or a new material his impression of beautiful
things.

The highest as the lowest form o


In [14]:

print(len(chs))

80


In [15]:
vocabSize = len(chs)

In [16]:
#tokenizer
'''char_to_int = {char: idx for idx, char in enumerate(chs)}
int_to_char = {idx: char for idx, char in enumerate(chs)}
def encode(s):
    return [char_to_int[char] for char in s]
def decode(int_list):
    return ''.join([int_to_char[idx] for idx in int_list])

encodedHello = encode('hello')
decodedHello = decode(encodedHello)
print(decodedHello)'''

"char_to_int = {char: idx for idx, char in enumerate(chs)}\nint_to_char = {idx: char for idx, char in enumerate(chs)}\ndef encode(s):\n    return [char_to_int[char] for char in s]\ndef decode(int_list):\n    return ''.join([int_to_char[idx] for idx in int_list])\n\nencodedHello = encode('hello')\ndecodedHello = decode(encodedHello)\nprint(decodedHello)"

In [17]:
#using pytorch ML framework
char_to_int = {char: idx for idx, char in enumerate(chs)}
int_to_char = {idx: char for idx, char in enumerate(chs)}
def encode(s):
    return [char_to_int[char] for char in s]
def decode(int_list):
    return ''.join([int_to_char[idx] for idx in int_list])

data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])

tensor([ 0, 33, 48, 45,  1, 29, 49, 43, 60, 61, 58, 45,  1, 55, 46,  1, 17, 55,
        58, 49, 41, 54,  1, 20, 58, 41, 65,  0,  0, 42, 65,  1, 28, 59, 43, 41,
        58,  1, 36, 49, 52, 44, 45,  0,  0,  0, 33, 21, 18,  1, 29, 31, 18, 19,
        14, 16, 18,  0,  0,  0, 33, 48, 45,  1, 41, 58, 60, 49, 59, 60,  1, 49,
        59,  1, 60, 48, 45,  1, 43, 58, 45, 41, 60, 55, 58,  1, 55, 46,  1, 42,
        45, 41, 61, 60, 49, 46, 61, 52,  1, 60])


In [18]:
#splitting
n = int(0.8*len(data))
trainData = data[:n]
testData = data[n:]
'''def getRandomChunk(split):
    filename = "/content/drive/MyDrive/output_train.txt" if split == 'train' else "/content/drive/MyDrive/output_val.txt"
    with open(filename, 'rb') as f: #opening in binary mode
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            fileSize = len(mm)
            startPos = random.randint(0, (fileSize) - blockSize*batchSize)

            # Seek to the random position and read the block of text
            mm.seek(startPos)
            block = mm.read(blockSize*batchSize-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decodedBlock = block.decode('utf-8', errors='ignore').replace('\r', '')

            # Train and test splits
            data = torch.tensor(encode(decodedBlock), dtype=torch.long)

    return data'''


def getBatch(split):
  #data = getRandomChunk(split)
  idx = torch.randint(len(data) - blockSize, (batchSize,)) #batchsize is shape
  #print(idx)
  x = torch.stack([data[i:i+blockSize] for i in idx])
  y = torch.stack([data[i+1:i+blockSize+1] for i in idx])
  x, y = x.to(device), y.to(device)
  return x,y

x, y = getBatch("train")
print("inputs:")
print(x)
print("targets:")
print(y)




inputs:
tensor([[49, 60, 48,  ..., 44, 49, 53],
        [54, 46, 55,  ..., 58, 55, 61],
        [ 1, 61, 59,  ..., 61, 47, 48],
        ...,
        [ 1, 48, 41,  ..., 48, 45,  1],
        [45,  1, 53,  ..., 53, 59, 45],
        [ 3,  1, 60,  ..., 42, 61, 60]], device='cuda:0')
targets:
tensor([[60, 48,  1,  ..., 49, 53,  1],
        [46, 55, 58,  ..., 55, 61, 47],
        [61, 59,  1,  ..., 47, 48, 60],
        ...,
        [48, 41, 44,  ..., 45,  1, 63],
        [ 1, 53, 41,  ..., 59, 45, 52],
        [ 1, 60, 61,  ..., 61, 60, 52]], device='cuda:0')


In [19]:
#bigram - knows current caharacter and just predicts the next one
x = trainData[:blockSize]
y = trainData[1:blockSize+1]

for z in range(blockSize):  #for loop to show the target and predictions
    context = x[:z+1]
    target = y[z]
    print("context :", context, "target :", target) #sequential'''

context : tensor([0]) target : tensor(33)
context : tensor([ 0, 33]) target : tensor(48)
context : tensor([ 0, 33, 48]) target : tensor(45)
context : tensor([ 0, 33, 48, 45]) target : tensor(1)
context : tensor([ 0, 33, 48, 45,  1]) target : tensor(29)
context : tensor([ 0, 33, 48, 45,  1, 29]) target : tensor(49)
context : tensor([ 0, 33, 48, 45,  1, 29, 49]) target : tensor(43)
context : tensor([ 0, 33, 48, 45,  1, 29, 49, 43]) target : tensor(60)
context : tensor([ 0, 33, 48, 45,  1, 29, 49, 43, 60]) target : tensor(61)
context : tensor([ 0, 33, 48, 45,  1, 29, 49, 43, 60, 61]) target : tensor(58)
context : tensor([ 0, 33, 48, 45,  1, 29, 49, 43, 60, 61, 58]) target : tensor(45)
context : tensor([ 0, 33, 48, 45,  1, 29, 49, 43, 60, 61, 58, 45]) target : tensor(1)
context : tensor([ 0, 33, 48, 45,  1, 29, 49, 43, 60, 61, 58, 45,  1]) target : tensor(55)
context : tensor([ 0, 33, 48, 45,  1, 29, 49, 43, 60, 61, 58, 45,  1, 55]) target : tensor(46)
context : tensor([ 0, 33, 48, 45,  1,

In [20]:
print(len(data))

429014


In [21]:
@torch.no_grad()
def estimateLoss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(evalIters)
    for k in range(evalIters):
      X, Y = getBatch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [22]:
#initializing neural networks
class Head(nn.Module):
  """one head of self-attenton"""
  def __init__(self, headSize):
    super().__init__()
    self.key = nn.Linear(nEmbd, headSize, bias = False) #converting 394 to 96
    self.query = nn.Linear(nEmbd, headSize, bias = False)
    self.value = nn.Linear(nEmbd, headSize, bias = False)
    self.register_buffer('tril', torch.tril(torch.ones(blockSize, blockSize)))
    #self.tril = nn.Parameter(torch.tril(torch.ones(blockSize, blockSize)).view(1, blockSize, blockSize), requires_grad=False)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    #input of size (batch, time-step, channels)
    #output of size( batch, time-step, headsize)
    B,T,C = x.shape #breaking the x
    k = self.key(x) # changing channel to headsize -> (B,T,C) becomes (B,T,hs)
    q = self.query(x) #  (B,T,C) becomes (B,T,hs)
    #compute attention scores("affinities")
    wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 #(B, T, hs) @ (B, hs, T) -> (B, T, T)
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) #(B, T, T)
    wei = F.softmax(wei, dim = -1) # exponentiates the masked weights to make the model more confident to score better in long run when we highlight what tokens and attention scores are more important
    wei = self.dropout(wei)
    #perform final matrix mul for weighted aggregation of values
    v = self.value(x) #(B, T, hs)
    out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
    return out

class MultiHeadAttention(nn.Module):
  """multiple heads of self attention in parallel"""
  def __init__(self, numHeads, headSize):
    super().__init__()
    self.heads = nn.ModuleList([Head(headSize) for _ in range(numHeads)]) #4 heads running in parallel
    self.proj = nn.Linear(headSize*numHeads, nEmbd) #if we wnat to change it, it wont throw dimensionality errors if we mention speciically, adds another learnable parameter
    self.dropout = nn.Dropout(dropout) #drops 20% of network neurons

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim = -1) #concatenate each head together along last dimension, last dimension is #(B, T, C), so we will concatenate by channel C also known as(feature) dimension
    out = self.dropout(self.proj(out))
    return out


class FeedForward(nn.Module):
  """a simple linear layer followed by a non-linearity"""

  def __init__(self, nEmbd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(nEmbd, 4*nEmbd),
        nn.ReLU(),
        nn.Linear(4*nEmbd, nEmbd), #output shape will be nEmbd*nEmbd
        nn.Dropout(dropout), #makes some neurons drop out and become zero to prevent overfitting

    )

  def forward(self, x):
    return self.net(x)


class Block(nn.Module):
  """Transformer block: communication followed by computation"""

  def __init__(self, nEmbd, n_head): #nEmbd = embedding dimension, n_head - the number of heads we would like
    super().__init__() #initializing things we do in the forward pass
    headSize = nEmbd//n_head #how many features is ecah head capturing = 394/4 = 96
    self.sa = MultiHeadAttention(n_head, headSize)# self attention
    self.ffwd = FeedForward(nEmbd)#linear,relu,linear
    self.ln1 = nn.LayerNorm(nEmbd)#postnorm
    self.ln2 = nn.LayerNorm(nEmbd)

  def forward(self, x):
    y = self.sa(x) #slef attention
    x = self.ln1(x+y) #add and norm
    y = self.ffwd(x) #forward
    x = self.ln2(x+y)#add and norm
    return x

class GPTLanguageModel(nn.Module):
  def __init__(self, vocabSize):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocabSize, nEmbd) #kindof like a heat map giving the probability of a character coming next to another, we can claculate the propotions by normalizing the character number in the table by the sum entire line
    self.position_embedding_table = nn.Embedding(blockSize, nEmbd)
    self.blocks = nn.Sequential(*[Block(nEmbd, n_head = n_head) for _ in range(nLayer)]) #builds 4 layers of decoders
    self.ln_f = nn.LayerNorm(nEmbd) #final layer norm (nn.Linear at end of decoder)
    self.lm_head = nn.Linear(nEmbd, vocabSize)
    self.apply(self._init_weights)

  def _init_weights(self, module): #initilization on weights
    if isinstance(module, nn.Linear):
      torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
      if module.bias is not None:
        torch.nn.init.zeros_(module.bias)
    elif isinstance(module, nn.Embedding):
      torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)

  def forward(self, index, targets=None): #writing a forward function helps in debugging and understanding behind the scenes of transformations
    B, T = index.shape
    #logits = self.token_embedding_table(index) #normalizing floating point numbers to get proportions of what comes next
    #B, T, C = logits.shape # to understand logits shape, by B - BATCH, by T - TIME(sequence of integers, we dont know next token is so thats why we say it time, cause we dont know next), by C - channel(vocabsize), so logits shape is B*T*C


    #idx and targets are both (B,T) tensor of integers
    #B, T, C = logits.shape
    tokEmb = self.token_embedding_table(index) #(B,T,C)
    posEmb = self.position_embedding_table(torch.arange(T, device = device)) #(T,C), lookup table
    x = tokEmb + posEmb #(B, T, C)
    x = self.blocks(x) #(B, T, C)
    x = self.ln_f(x) #(B, T, C)
    logits = self.lm_head(x) #(B, T, vocabSize)

    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T, C) # blending Batch and Time together, since they are not as significant as batch, as long as logits and targets have the same batch in time, it should work
      targets = targets.view(B*T) #view function is used to change shape, to unpack with .shape and repack with .view
      loss = F.cross_entropy(logits, targets) #loss function, using entropy to measure it
    return logits, loss

  def generate(self, index, maxNewTokens) :
    for _ in range(maxNewTokens):
      logits, loss = self.forward(index)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim = -1)
      indexNext = torch.multinomial(probs, num_samples=1)
      index = torch.cat((index, indexNext), dim = 1)
    return index

model = GPTLanguageModel(vocabSize)

'''with open('model1.pkl', 'rb') as f:
  model = pickle.load(f)
print("loaded parameters")'''
m = model.to(device)

'''context = torch.zeros((1,1), dtype = torch.long, device = device)
generatedChars = decode(m.generate(context, maxNewTokens = 500)[0].tolist())
print(generatedChars)'''



'context = torch.zeros((1,1), dtype = torch.long, device = device)\ngeneratedChars = decode(m.generate(context, maxNewTokens = 500)[0].tolist())\nprint(generatedChars)'

In [23]:
#view example

#a = torch.rand(2,3,5)
#x, y, z = a.shape # x= 2, y=3, z=5
#a = a.view(x, y, z) #making tensor 'a' again with those dimensions
#print(a.shape)

#so we can unpack the tensor with .shape with use .view to put it together

In [ ]:
#create a pytorch Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = learningRate)
for iter in range(maxIters): #training loop
  if iter %evalIters == 0:
    losses = estimateLoss()
    print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}") # to understand the losses for every couple of iterations to see if the model is actually doing better
  xb, yb = getBatch('train') #inputs, targets

  logits, loss = model.forward(xb, yb) #forward pass
  optimizer.zero_grad(set_to_none=True) #define optimizer
  loss.backward() #backward pass
  optimizer.step() #gradient descent
print(loss.item())

#with open('model1.pkl', 'wb') as f:
 # pickle.dump(model, f)
#default pytorch accumualtes the gradient by adding them
#overtime but using zero gradient, set to non true, will not add the
#previous gradients, so the previous gradients do not effect
#the current one

step: 0, train loss: 4.467, val loss: 4.466
step: 100, train loss: 2.311, val loss: 2.314
step: 200, train loss: 1.954, val loss: 1.951


In [ ]:
'''from google.colab import files
files.download('model1.pkl')'''


In [ ]:
'''#relu example
x = torch.tensor([0.05], dtype = torch.float32)
y = F.relu(x)
print(y)

In [ ]:
'''#sigmoid example
x = torch.tensor([-0.05], dtype = torch.float32)
y = F.sigmoid(x)
print(y)

In [ ]:
'''#tanh example
x = torch.tensor([1], dtype = torch.float32)
y = F.tanh(x)
print(y)